# Python

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

In [ ]:
%%shell
mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

# R

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if(!require("R.utils")) install.packages("R.utils", repos = "http://cran.us.r-project.org")
require("R.utils")

In [ ]:
require("data.table")

# 1. Definición de archivos
archivos_prob <- c(
  "/content/buckets/b1/exp/Comp3_Entrega-400/prediccion.txt",
  "/content/buckets/b1/exp/Comp3_Entrega-300/prediccion.txt"
)

# 2. Carga y Ensemble (Soft Voting)
tbl_probs <- rbindlist(lapply(archivos_prob, function(x) {
  fread(x, select = c("numero_de_cliente", "prob"))
}))

ensemble_final <- tbl_probs[, .(prob_promedio = mean(prob)), by = numero_de_cliente]

# 3. Ordenamiento y Corte
setorder(ensemble_final, -prob_promedio) # Orden descendente

corte_seleccionado <- 11250

# 4. Filtrado y Selección
# Nos quedamos SOLAMENTE con los primeros 11250 registros (Top del ranking)
# Y seleccionamos SOLO la columna numero_de_cliente
lista_ids <- ensemble_final[1:corte_seleccionado, .(numero_de_cliente)]

# 5. Guardado (Sin encabezados)
archivo_salida <- "/content/buckets/b1/exp/Comp3_DuilioCamps_Lista_IDs_Corte11250.csv" # O .txt si prefieres

fwrite(lista_ids,
       file = archivo_salida,
       col.names = FALSE, # <--- ESTO quita el encabezado
       sep = "\n"         # Un ID por línea (seguro para listas)
)

# 6. Validación
cat("=== ARCHIVO DE IDs GENERADO ===\n")
cat("Ubicación:", archivo_salida, "\n")
cat("Cantidad de IDs guardados:", nrow(lista_ids), "\n")
cat("¿Tiene encabezados?: NO\n")
cat("Ejemplo de los primeros 3 IDs:\n")
print(head(lista_ids, 3))